In [81]:

import numpy as np
import pandas as pd
import gzip
import scipy.stats as stats
import argparse
import os
from gtex_normalization import normalize_expression


def get_donors(path):
    donor_ids = list()
    with open(path, 'r') as instream:
        next(instream)
        next(instream)
        for line in instream:
            donor_ids.append(line.strip().split()[0])
    return donor_ids

def read_gct(gct_file, donor_ids=None):
    """
    Load GCT as DataFrame
    """    
    df = pd.read_csv(gct_file, sep='\t', skiprows=2, index_col=0)
    df.drop('Description', axis=1, inplace=True)
    df.index.name = 'gene_id'
    if donor_ids is not None:
        df = df[[i for i in df.columns if '-'.join(i.split('-')[:2]) in donor_ids]]
    return df

In [55]:
path = "/data/franco/datasets/gtex_v8/expression"
expression_gct = os.path.join(path, "GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct.gz") # "rpkm file"
counts_gct = os.path.join(path, "GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_reads.gct.gz")    # file with read counts
# expression_threshold=0.1    # 'Selects genes with > expression_threshold expression in at least min_samples')
# count_threshold=5,          # 'Selects genes with > count_threshold reads in at least min_samples')
# min_samples=10              # 'Minimum number of samples that must satisfy thresholds')
pheno_file = os.path.join(path, "../phenotypes/phs000424.v8.pht002743.v8.p2.c1.GTEx_Sample_Attributes.GRU.txt")
donor_file = os.path.join(path, "../genotypes/gtex_v8.sample")

In [56]:
# Load genotype donor ids, we will use them to filter rna-seq samples that also have genotype
pheno_df = pd.read_csv(pheno_file, sep="\t", comment="#", header=0)
gt_donors = get_donors(donor_file)

In [99]:
# pheno_df[pheno_df['SMGEBTCHT'].str.contains("TruSeq.v1")]['SAMPID']]

In [34]:
# Get list of all tissues present in the phenotype file
tissue_names = list(pheno_df[pheno_df['SMGEBTCHT'].str.contains("TruSeq.v1")]["SMTSD"].unique())

In [74]:
# From the phenotype file loaded above, filter RNA-seq samples that are now flagged and belong to each tissue
def get_samples(pheno_df, tissue, gt_donors):
    sub_df = pheno_df.loc[(pheno_df['SMTORMVE'] != "FLAGGED") & (pheno_df['SMGEBTCHT'] == "TruSeq.v1") & (pheno_df['SMTSD'] == tissue)]
    valid_donors = [i for i in list(sub_df["SAMPID"]) if '-'.join(i.split('-')[:2]) in gt_donors]
    # print([True for i in list(sub_df["SAMPID"]) if i in valid_donors])
    return valid_donors
    
donors_per_tissue = dict()
for tissue in tissue_names:
    donors_per_tissue[tissue] = get_samples(pheno_df, tissue, gt_donors)

In [100]:
# Count for each tissue, the number of samples with Genotype, should match gtexportal
# Filter tissues with less than 30 samples
counter = 1
for t in donors_per_tissue:
    if len(donors_per_tissue[t]) > 30:
        print(counter, t, len(donors_per_tissue[t]))
        counter += 1

1 Adipose - Subcutaneous 581
2 Muscle - Skeletal 706
3 Artery - Tibial 584
4 Artery - Coronary 213
5 Heart - Atrial Appendage 372
6 Adipose - Visceral (Omentum) 469
7 Ovary 167
8 Uterus 129
9 Vagina 141
10 Breast - Mammary Tissue 396
11 Skin - Not Sun Exposed (Suprapubic) 517
12 Minor Salivary Gland 144
13 Brain - Cortex 205
14 Adrenal Gland 233
15 Thyroid 574
16 Lung 515
17 Spleen 227
18 Pancreas 305
19 Esophagus - Muscularis 465
20 Esophagus - Mucosa 497
21 Esophagus - Gastroesophageal Junction 330
22 Stomach 324
23 Colon - Sigmoid 318
24 Small Intestine - Terminal Ileum 174
25 Colon - Transverse 368
26 Prostate 221
27 Testis 322
28 Skin - Sun Exposed (Lower leg) 605
29 Nerve - Tibial 532
30 Heart - Left Ventricle 386
31 Pituitary 237
32 Brain - Cerebellum 209
33 Cells - Cultured fibroblasts 483
34 Whole Blood 670
35 Artery - Aorta 387
36 Cells - EBV-transformed lymphocytes 147
37 Brain - Frontal Cortex (BA9) 175
38 Brain - Cerebellar Hemisphere 175
39 Brain - Caudate (basal ganglia)

In [82]:
expression_df = read_gct(expression_gct, gt_donors) #

In [83]:
expression_df.shape

(56200, 15253)

In [93]:
def write_gct(df, filepath, trim_ids=True, header=False):
    """
    Write dataframe as a GCT file
    """
    with open(filepath, 'w') as mfile:
        if header:
            mfile.write("#1.2\n")
            mfile.write('%i\t%i\n' % (df.shape[0], df.shape[1] - 1))
        # mfile.write(str(df.shape[0])+'\t'+str(df.shape[1] - 1)+'\n')
        if trim_ids:
            new_headers = ['-'.join(i.split('-')[:2]) for i in df.columns]
            df.columns = new_headers
        df.to_csv(mfile, sep='\t', index=True, header=True)

In [101]:
# create a dictionary from the whole name to the acronyms
names_df = pd.read_table("/data/franco/datasets/gtex_v8/tissues_list.txt", sep="\t", comment="#", header=None)
names_df.columns = ["long_name", "acr", "id", "npeer"]
names2acr = dict(zip(names_df["long_name"], names_df["acr"]))

In [103]:
for t in tissue_names:
    if len(donors_per_tissue[t]) > 30 and t in names2acr:
        print(f"Processing {t} -> {names2acr[t]}")
        outfile = f"/data/franco/datasets/gtex_v8/expression/GTEx_tpms/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9.gene_tpm.{names2acr[t]}.txt"
        if not os.path.exists(outfile):
            expr_tissue_df = expression_df[donors_per_tissue[t]]
            write_gct(expr_tissue_df, outfile)
        else:
            print(f"{t} file exists")
    else:
        print("ERROR: tissue not in table or <30 samples")

Processing Adipose - Subcutaneous -> as
Adipose - Subcutaneous file exists
Processing Muscle - Skeletal -> ms
Muscle - Skeletal file exists
Processing Artery - Tibial -> at
Artery - Tibial file exists
Processing Artery - Coronary -> ac
Artery - Coronary file exists
Processing Heart - Atrial Appendage -> haa
Heart - Atrial Appendage file exists
Processing Adipose - Visceral (Omentum) -> av
Adipose - Visceral (Omentum) file exists
Processing Ovary -> ov
Ovary file exists
Processing Uterus -> ut
Uterus file exists
Processing Vagina -> va
Vagina file exists
Processing Breast - Mammary Tissue -> br
Breast - Mammary Tissue file exists
Processing Skin - Not Sun Exposed (Suprapubic) -> snse
Skin - Not Sun Exposed (Suprapubic) file exists
Processing Minor Salivary Gland -> msg
Minor Salivary Gland file exists
Processing Brain - Cortex -> bco
Brain - Cortex file exists
Processing Adrenal Gland -> ag
Adrenal Gland file exists
Processing Thyroid -> thy
Thyroid file exists
Processing Lung -> lu
Lun

In [104]:
del expression_df

In [105]:
counts_df = read_gct(counts_gct, gt_donors)

In [106]:
counts_df.shape

(56200, 15253)

In [107]:
for t in tissue_names:
    if len(donors_per_tissue[t]) > 30 and t in names2acr:
        print(f"Processing {t} -> {names2acr[t]}")
        outfile = f"/data/franco/datasets/gtex_v8/expression/GTEx_reads/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9.gene_reads.{names2acr[t]}.txt"
        if not os.path.exists(outfile):
            counts_tissue_df = counts_df[donors_per_tissue[t]]
            write_gct(counts_tissue_df, outfile)
        else:
            print(f"{t} file exists")
    else:
        print("ERROR: tissue not in table or <30 samples")

Processing Adipose - Subcutaneous -> as
Processing Muscle - Skeletal -> ms
Processing Artery - Tibial -> at
Processing Artery - Coronary -> ac
Processing Heart - Atrial Appendage -> haa
Processing Adipose - Visceral (Omentum) -> av
Processing Ovary -> ov
Processing Uterus -> ut
Processing Vagina -> va
Processing Breast - Mammary Tissue -> br
Processing Skin - Not Sun Exposed (Suprapubic) -> snse
Processing Minor Salivary Gland -> msg
Processing Brain - Cortex -> bco
Processing Adrenal Gland -> ag
Processing Thyroid -> thy
Processing Lung -> lu
Processing Spleen -> spl
Processing Pancreas -> pan
Processing Esophagus - Muscularis -> esomu
Processing Esophagus - Mucosa -> esom
Processing Esophagus - Gastroesophageal Junction -> esog
Processing Stomach -> sto
Processing Colon - Sigmoid -> cols
Processing Small Intestine - Terminal Ileum -> si
Processing Colon - Transverse -> colt
Processing Prostate -> pro
Processing Testis -> tes
Processing Skin - Sun Exposed (Lower leg) -> sse
Processing

In [31]:

# print('Normalizing using all genes within %i samples ...' % expression_df.shape[1])
# quant_std_df, quant_df = normalize_expression(wblood_expression_df, wblood_counts_df,
#     expression_threshold=expression_threshold, count_threshold=count_threshold, min_samples=min_samples)


Normalizing using all genes within 8116 samples ...
